In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import pandas as pd
import nibabel as nib
import segmentation_models_pytorch as smp
import copy
import time
import torchvision.models as models
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import albumentations as albu
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
def get_data():
    
    df = pd.read_csv('Rectal gas project.csv')
    files_images = os.listdir('images')
    files_mask = os.listdir('GT')

    X = np.zeros((211, 512, 512, 7), dtype='float32')
    X_masks = np.zeros((211, 512, 512, 7), dtype='float32')
    Y = np.zeros((211,), dtype='uint8')

    index = 0
    for i in tqdm(range(len(files_images))):

        img = nib.load(f'images/M{i+1}.nii').get_fdata()
        mask = nib.load(f'GT/M{i+1}SEG.nii').get_fdata()
        
        if img.shape != (7, 512, 512):
            if img.shape == (512, 512, 7):
                X[index] = img
                X_masks[index] = mask
                index += 1
            continue
        
        X[index] = np.moveaxis(img, 0, -1)
        X_masks[index] = np.moveaxis(mask, 0, -1)
            
        name = f'M{i+1}'
        Y[index] = int(df.loc[df['Case'] == name]['Rectal content'])
        index += 1
    return X, X_masks, Y

In [3]:
images, masks, labels = get_data()

100%|██████████| 213/213 [00:21<00:00, 10.14it/s]


In [4]:
images = np.moveaxis(images, -1, 1)
images = images/np.max(images)
images = images*(images > 0)

masks = np.moveaxis(masks, -1, 1)
masks = (masks>0).astype('float32')

In [5]:
images_train, images_test, masks_train, masks_test, labels_train, labels_test = train_test_split(images, masks, labels, test_size=0.3, random_state=42)

In [6]:
masks = np.concatenate((masks_train, masks_test), axis=0)
images = np.concatenate((images_train, images_test), axis=0)
labels = np.concatenate((labels_train, labels_test), axis=0)

In [7]:
def dice_coef(y_true, y_pred):
    smooth = 1e-6
    y_true_f = torch.flatten(y_true)
    y_pred_f = torch.flatten(y_pred)
    intersection = torch.sum(y_true_f * y_pred_f)
    return (2.*intersection + smooth)/(torch.sum(y_true_f) + torch.sum(y_pred_f) + smooth)

In [8]:
def get_dice_score(file, model, device='cuda'):
    
    model.load_state_dict(torch.load(file))
    model = model.to(device)
    masks_pred_train = np.zeros((images_train.shape[0], 7, 512, 512))
    masks_pred_test = np.zeros((images_test.shape[0], 7, 512, 512))

    for i in tqdm(range(images_train.shape[0])):
        outputs = model(torch.Tensor(images_train[i]).unsqueeze(1).to(device))
        outputs = outputs.squeeze(1)
        outputs = outputs.detach().cpu().numpy()
        outputs = (outputs > 0.5).astype('float32')
        masks_pred_train[i] = outputs

    for i in tqdm(range(images_test.shape[0])):
        outputs = model(torch.Tensor(images_test[i]).unsqueeze(1).to(device))
        outputs = outputs.squeeze(1)
        outputs = outputs.detach().cpu().numpy()
        outputs = (outputs > 0.5).astype('float32')
        masks_pred_test[i] = outputs

    
    masks_pred = np.concatenate((masks_pred_train, masks_pred_test), axis=0)
    
    dice_train = dice_coef(torch.Tensor(masks_train), torch.Tensor(masks_pred_train))
    print(f'Train Dice Score: {dice_train}')

    dice_test = dice_coef(torch.Tensor(masks_test), torch.Tensor(masks_pred_test))
    print(f'Test Dice Score: {dice_test}')

    dice_total = dice_coef(torch.Tensor(masks), torch.Tensor(masks_pred))
    print(f'Total Dice Score: {dice_total}')

# Get Dice Scores from all the trained models

In [9]:
# Unet++ Resnet34 Imagenet

file = 'unetplusplus_mri_loc_resnet34_imagenet_1.pt'
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.UnetPlusPlus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1,
    encoder_depth=5
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:11<00:00,  5.70it/s]


Train Dice Score: 0.9392305016517639
Test Dice Score: 0.8405038714408875
Total Dice Score: 0.9089540243148804


In [10]:
# Unet Resnet50 Imagenet

file = 'unet_mri_loc_resnet50_imagenet_1.pt'
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1,
    encoder_depth=5
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:08<00:00,  7.96it/s]


Train Dice Score: 0.9169445037841797
Test Dice Score: 0.8408658504486084
Total Dice Score: 0.8935545682907104


In [11]:
# Unet Resnet34 Without Imagenet

file = 'unet_mri_loc_resnet34_1.pt'
ENCODER = 'resnet34'
ENCODER_WEIGHTS = None
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:05<00:00, 11.81it/s]


Train Dice Score: 0.883194088935852
Test Dice Score: 0.8270816802978516
Total Dice Score: 0.865754246711731


In [12]:
# Unet Resnet34 With Imagenet 1

file = 'unet_mri_loc_resnet34_imagenet_1.pt'
ENCODER_WEIGHTS = 'imagenet'
ENCODER = 'resnet34'
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:05<00:00, 11.60it/s]


Train Dice Score: 0.880196213722229
Test Dice Score: 0.8410815596580505
Total Dice Score: 0.8681454062461853


In [13]:
# Unet Resnet34 With Imagenet 2

file = 'unet_mri_loc_resnet34_imagenet_2.pt'
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:05<00:00, 11.79it/s]


Train Dice Score: 0.927670955657959
Test Dice Score: 0.8499149084091187
Total Dice Score: 0.9037346243858337


In [14]:
# Unet VGG16 With Imagenet 1

file = 'unet_mri_loc_vgg16_imagenet_1.pt'
ENCODER = 'vgg16'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:12<00:00,  5.30it/s]


Train Dice Score: 0.8750020265579224
Test Dice Score: 0.8394221067428589
Total Dice Score: 0.864011287689209


In [15]:
# Unet VGG16 With Imagenet 2

file = 'unet_mri_loc_vgg16_imagenet_2.pt'
ENCODER = 'vgg16'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['tissue']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=1
)

get_dice_score(file, model)

100%|██████████| 64/64 [00:11<00:00,  5.34it/s]


Train Dice Score: 0.9085118770599365
Test Dice Score: 0.8359131813049316
Total Dice Score: 0.8858718276023865
